In [1]:
import argparse
import sys

def parse_args():
    # 在 Jupyter 环境下，跳过 argparse 解析
    if 'ipykernel_launcher' in sys.argv[0]:
        sys.argv = sys.argv[:1]  # 只保留脚本名，不处理后续参数

    parser = argparse.ArgumentParser(description="Your script description")
    parser.add_argument('--outf', type=str, help="Output file path")
    args = parser.parse_args()
    return args

args = parse_args()
print(args)

Namespace(outf=None)


In [2]:
import pickle
import numpy as np

def load_batch(file_path):
    """从文件中加载一个批次的数据"""
    with open(file_path, 'rb') as f:
        batch = pickle.load(f, encoding='bytes')
        data = batch[b'data']
        labels = batch[b'labels']
        return data, labels

# 加载所有批次
data_list = []
labels_list = []
for i in range(1, 6):  # 5个训练批次
    file_path = f'./data/cifar-10-batches-py/data_batch_{i}'
    data, labels = load_batch(file_path)
    data_list.append(data)
    labels_list.append(labels)

# 合并所有批次的数据
train_data = np.concatenate(data_list, axis=0)
train_labels = np.concatenate(labels_list, axis=0)

print(f"训练数据形状: {train_data.shape}")  # (50000, 3072)
print(f"训练标签形状: {train_labels.shape}")  # (50000,)

训练数据形状: (50000, 3072)
训练标签形状: (50000,)


In [3]:
import torch
from torchvision import transforms

# 假设 train_data 是 numpy 数组，train_labels 是标签数组
# 将 train_data 转换为 PyTorch 张量并调整形状为 (N, C, H, W)
train_data = torch.tensor(train_data, dtype=torch.float32).reshape(-1, 3, 32, 32) / 255.0
train_labels = torch.tensor(train_labels, dtype=torch.long)

# 定义数据预处理（Normalize 按通道归一化）
transform = transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))

# 应用归一化（注意数据格式应为 [N, C, H, W]）
train_data = transform(train_data)

In [5]:
import os
import torch
from torch.utils.data import TensorDataset, DataLoader
from resnet import ResNet18  # 确保您实现或导入了 ResNet18
from tqdm import tqdm  # 用于显示进度条

# 定义设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 构造训练集 DataLoader
# 假设 train_data 和 train_labels 已定义并正确处理
train_dataset = TensorDataset(train_data, train_labels)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=False)

# 模型文件路径
model_path = './model/net_197.pth'

# 检查模型文件是否存在
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found: {model_path}")

# 加载模型
net = ResNet18().to(device)  # 将模型移动到设备
try:
    net.load_state_dict(torch.load(model_path, map_location=device))
    net.eval()  # 设置为评估模式
    print(f"Model loaded successfully from {model_path}")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# 计算训练集准确率
correct = 0
total = 0

print("Calculating training set accuracy...")
with torch.no_grad():  # 禁用梯度计算
    for inputs, labels in tqdm(train_loader, desc="Processing Batches"):
        # 将数据移动到设备
        inputs, labels = inputs.to(device), labels.to(device)

        # 模型前向传播
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)  # 获取预测类别

        # 累计正确数量和总数量
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 计算准确率
train_acc = 100. * correct / total
print(f"训练集准确率: {train_acc:.2f}%")

Using device: cuda
Model loaded successfully from ./model/net_197.pth
Calculating training set accuracy...


Processing Batches: 100%|██████████| 500/500 [00:07<00:00, 70.97it/s]

训练集准确率: 98.77%
